In [ ]:
import numpy as np

import theano.tensor as TT
import time
import random

import pymc3 as pm
import arviz as az

## Defino las funciones que voy a usar en el esquema bayesiano para ajustar la señal reflejada por el target.

In [ ]:
## solo un medio dieléctrico

def toFit_1L(ep):
    
    S1 = np.abs(Scal*R_ep(ep))
  
    # return np.asarray([S1_real, S1_imag])
    dim = int(np.shape(Scal)[0])
 
    a = TT.zeros((1,dim))
    a1 = TT.set_subtensor(a[0], S1)
 
    return a1
##----------------------------------------------------------------------##

## un medio dieléctrico con ep1 y ancho d, luego un medio semi-infinito con ep2

def toFit_2L(ep1,ep2,d):
    
    S1 = np.abs(Scal*R_ep1_ep2(ep1,ep2,d,fpos))
    
  
    # return np.asarray([S1_real, S1_imag])
    dim = int(np.shape(Scal)[0])
 
    a = TT.zeros((1,dim))
    a1 = TT.set_subtensor(a[0], S1)
 
    return a1


def toFit_2L_rug(ep1,ep2,d,s0,s1):
    
    S1 = np.abs(Scal*R_pinel_2L(ep1,ep2,d,s0,s1,fpos))
  
    # return np.asarray([S1_real, S1_imag])
    dim = int(np.shape(Scal)[0])
    
    a = TT.zeros((1,dim))
    a1 = TT.set_subtensor(a[0], S1)
    
    return a1
    


##----------------------------------------------------------------------##

## un medio con ep1 y ancho d1, luego uno con ep2 y ancho d2, luego uno semi-infinito con ep3

def toFit_3L(ep1,ep2,ep3,d1,d2):
    
    S1 = np.abs(Scal*R_ep1_ep2_ep3(ep1, ep2, ep3, d1, d2, fpos))
  
    # return np.asarray([S1_real, S1_imag])
    dim = int(np.shape(Scal)[0])
 
    a = TT.zeros((1,dim))
    a1 = TT.set_subtensor(a[0], S1)
 
    return a1


def toFit_3L_rug_3L(ep1,ep2,ep3,d1,d2,s0,s1,s2):
    
    S1 = np.abs(Scal*R_pinel(ep1,ep2,ep3,d1,d2,s0,s1,s2,fpos))
  
    # return np.asarray([S1_real, S1_imag])
    dim = int(np.shape(Scal)[0])
    
    a = TT.zeros((1,dim))
    a1 = TT.set_subtensor(a[0], S1)
    
    return a1

## Función principal del esquema bayesiano

In [ ]:
def modelo(S2, samples, sigmas,n_layers,rug):
    
    '''
    S2: medición sobre el target de interés: tiene tamaño (2,n_frec);
    el 2 es porque separé parte Real e Imaginaria de la señal.
    samples: escalar, nro de muestras para la inferencia bayesiana.
    sigmas: varianza de la medición, de tamaño (2, n_frec).
    
    n_layers = nro de medios materiales a usar en el modelo teórico.
    
    rug = True o False: incluye o no efectos de rugosidad
    
    OBS:
        - S1 y S2 son complejos de dimensión N
        - frecs es un vector real de dimensión N
    '''
    
    ## tomo parte real e imaginaria de la señal a ajustar
    S2 = np.abs(S2)
    
    ## defino el número de capas del modelo
    if(n_layers == 1):
        
        ep1r_min, ep1r_max = 4, 30

        ranges = {}
        ranges['ep1_r'] = ep1r_min, ep1r_max
        
        with pm.Model() as model:
            ep1_r = pm.Uniform('ep1_r', ranges['ep1_r'][0], ranges['ep1_r'][1])
            #ep1_i = pm.Uniform('ep1_i', 0.25*ranges['ep1_r'][0], 6*ranges['ep1_r'][0])
            ep1_i = pm.Uniform('ep1_i', 3.0, 10)

        # def f(ep1_ = ep1_r + 1j*ep1_i ):
        def f(ep1r_ = ep1_r, 
              ep1i_ = ep1_i):
            return toFit_1L(ep1r_ + 1j*ep1i_)

        with model:
            function_pm = pm.Deterministic('S2_teo', f())
            observations = pm.Normal( 'S2_med',  mu = function_pm, sigma = sigmas, observed = np.asarray([S2]))
            trace = pm.sample_smc(samples, cores=6, parallel=True)
            #ppc = pm.sample_posterior_predictive(trace, var_names=["ep1_r","ep1_i","S2_med"], random_seed=4, samples=10)
            
        return trace#, ppc, model
        print(function_pm)
    
    
    elif(n_layers == 2):
        
        if(rug == False):
        
            ep1_min = 4
            ep1_max = 18
            ep2_max = 28

            ranges = {}
            ranges['ep1'] = ep1_min, ep1_max
            ranges['ep2'] = 0.75*ep1_max, ep2_max
            ranges['d'] = 1e-4, 9e-3

            with pm.Model() as model:
                ep1_r = pm.Uniform('ep1_r', ranges['ep1'][0], ranges['ep1'][1])
                #ep1_i = pm.Uniform('ep1_i', 4, 8)
                ep2_r = pm.Uniform('ep2_r', ranges['ep2'][0], ranges['ep2'][1])
                #ep2_i = pm.Uniform('ep2_i', 6, 11)
                d = pm.Uniform('d', ranges['d'][0], ranges['d'][1])
                poten = pm.Potential('const',pm.HalfNormal.dist(10).logp(ep2_r-ep1_r))


            def f(ep1r_ = ep1_r, ep1i_ = 5, 
                  ep2r_ = ep2_r, ep2i_ = 6, 
                  d_ = d):
                return toFit_2L(ep1r_+1j*ep1i_,ep2r_+1j*ep2i_,d_)

            with model:
                function_pm = pm.Deterministic('S2_teo', f())
                observations = pm.Normal( 'S2_med',  mu = function_pm, sigma = sigmas, observed = np.asarray([S2]))
                trace = pm.sample_smc(samples, cores=6, parallel=True)

            return trace
            print(function_pm)
        
        if(rug == True):
            
            ep1_min = 9
            ep1_max = 25
            ep2_max = 30

            ranges = {}
            ranges['ep1'] = ep1_min, ep1_max
            ranges['ep2'] = 0.75*ep1_max, ep2_max
            ranges['d'] = 1e-2, 1e-1
            ranges['s0'] = 1e-5, 2e-3
            ranges['s1'] = 1e-5, 2e-3

            with pm.Model() as model:
                ## armo los priors. genero variables aleatorias. no sè cuanto vale c/u
                ## solo tengo una manera de genrar muestras 
                ep1_r = pm.Uniform('ep1_r', ranges['ep1'][0], ranges['ep1'][1])
                #ep1_i = pm.Uniform('ep1_i', 2, 6)
                ep2_r = pm.Uniform('ep2_r', ranges['ep2'][0], ranges['ep2'][1])
                #ep2_i = pm.Uniform('ep2_i', 3, 8)
                d = pm.Uniform('d', ranges['d'][0], ranges['d'][1])
                s0 = pm.Uniform('s0', ranges['s0'][0], ranges['s0'][1])
                s1 = pm.Uniform('s1', ranges['s1'][0], ranges['s1'][1])
                
                ## doy restricciones entre variables: ep2_r > ep1_r y s0 > s1
                poten_ep = pm.Potential('const_ep',pm.HalfNormal.dist(10).logp(ep2_r-ep1_r))
                poten_rug = pm.Potential('const_rug',pm.HalfNormal.dist(10).logp(s0-s1))
                
            
            ## genero una función determinista (modelo de scattering) que depende de variables aleatorias (los parámetros a encotrar)

            def f(ep1r_ = ep1_r, ep1i_ = 5, 
                  ep2r_ = ep2_r, ep2i_ = 6,
                  d_ = d,
                  s0_ = s0,
                  s1_ = s1):
                return toFit_2L_rug(ep1r_+1j*ep1i_,ep2r_+1j*ep2i_,d_,s0_,s1_)
            
            ## con la función de scattering evaluada en las variables aleatorias genero la variable 
            ## function_pm que es el valor medio (mu) de la 'medición modelada'; sigma es la incerteza en el proceso de medición (electrónica, mecánica, etc);
            ## esa medición modelada la comparon con 'observed'(=dato observado) y lo hago a través de trace, porque
            ## doy un montón (Nro de muestras) de puntos para cada una de las variables aleatorias de entrada.
            ## las trazas son muuestras de la distribuciòn posterior; sin Nro de muestra ---> infinito, la traza ES la 
            ## distribución posterior: de ahì puedo leer valores medios, varianzasa, deciles, cuartiles, etc.

            with model:
                function_pm = pm.Deterministic('S2_teo', f())
                observations = pm.Normal( 'S2_med',  mu = function_pm, sigma = sigmas, observed = np.asarray([S2]))
                trace = pm.sample_smc(samples, cores=6, parallel=True)

            return trace
            print(function_pm)
            
            
            
    elif(n_layers == 3):
        
        if(rug==False):
            
               
            ep1_min = 16
            ep1_max = 22
            ep2_max = 28
            ep3_max = 35

            ranges = {}
            ranges['ep1'] = ep1_min, ep1_max
            ranges['ep2'] = 0.85*ep1_min, ep2_max
            ranges['ep3'] = 0.85*ep2_max, ep3_max
            ranges['d1'] = 1e-2, 4e-2
            ranges['d2'] = 6e-2, 15e-2

            with pm.Model() as model:
                ep1_r = pm.Uniform('ep1_r', ranges['ep1'][0], ranges['ep1'][1])
                ep1_i = pm.Uniform('ep1_i', 2, 6)
                ep2_r = pm.Uniform('ep2_r', ranges['ep2'][0], ranges['ep2'][1])
                ep2_i = pm.Uniform('ep2_i', 4, 8)
                ep3_r = pm.Uniform('ep3_r', ranges['ep3'][0], ranges['ep3'][1])
                ep3_i = pm.Uniform('ep3_i', 4, 10)

                d1 = pm.Uniform('d1', ranges['d1'][0], ranges['d1'][1])
                d2 = pm.Uniform('d2', ranges['d2'][0], ranges['d2'][1])


            def f(ep1r_ = ep1_r, ep1i_ = ep1_i, 
                  ep2r_ = ep2_r, ep2i_ = ep2_i,
                  ep3r_ = ep3_r, ep3i_ = ep3_i,
                  d1_ = d1,
                  d2_ = d2):
                return toFit_3L(ep1r_+1j*ep1i_,ep2r_+1j*ep2i_,ep3r_+1j*ep3i_,d1_,d2_)

            with model:
                function_pm = pm.Deterministic('S2_teo', f())
                observations = pm.Normal( 'S2_med',  mu = function_pm, sigma = sigmas, observed = np.asarray([S2]))
                trace = pm.sample_smc(samples, cores=6, parallel=True)

            return trace    
            print(function_pm)
            
        if(rug == True):
            
            ep1_min = 9
            ep1_max = 18
            ep2_min = 14
            ep2_max = 27
            ep3_min = 22
            ep3_max = 40

            ranges = {}
            ranges['ep1'] = ep1_min, ep1_max
            ranges['ep2'] = ep2_min, ep2_max
            ranges['ep3'] = ep3_min, ep3_max
            ranges['d1'] = 1e-2, 1e-1
            ranges['d2'] = 1e-1, 2.5e-1
            ranges['s0'] = 1e-5, 2e-3
            ranges['s1'] = 1e-5, 2e-3
            ranges['s2'] = 0.5e-5, 2e-3

            with pm.Model() as model:
                
                ep1_r = pm.Uniform('ep1_r', ranges['ep1'][0], ranges['ep1'][1])
                ep1_i = pm.Uniform('ep1_i', 2, 6)
                ep2_r = pm.Uniform('ep2_r', ranges['ep2'][0], ranges['ep2'][1])
                ep2_i = pm.Uniform('ep2_i', 3, 8)
                ep3_r = pm.Uniform('ep3_r', ranges['ep3'][0], ranges['ep3'][1])
                ep3_i = pm.Uniform('ep3_i', 3, 8)
                
                d1 = pm.Uniform('d1', ranges['d1'][0], ranges['d1'][1])
                d2 = pm.Uniform('d2', ranges['d2'][0], ranges['d2'][1])
                
                s0 = pm.Uniform('s0', ranges['s0'][0], ranges['s0'][1])
                s1 = pm.Uniform('s1', ranges['s1'][0], ranges['s1'][1])
                s2 = pm.Uniform('s2', ranges['s2'][0], ranges['s2'][1])


            def f(ep1r_ = ep1_r, ep1i_ = ep1_i, 
                  ep2r_ = ep2_r, ep2i_ = ep2_i,
                  ep3r_ = ep3_r, ep3i_ = ep3_i,
                  d1_ = d1,
                  d2_ = d2,
                  s0_ = s0,
                  s1_ = s1,
                  s2_ = s2):
                return toFit_3L_rug(ep1r_+1j*ep1i_,ep2r_+1j*ep2i_,ep3r_+1j*ep3i_,d1_,d2_,s0_,s1_,s2_)

            with model:
                function_pm = pm.Deterministic('S2_teo', f())
                observations = pm.Normal( 'S2_med',  mu = function_pm, sigma = sigmas, observed = np.asarray([S2]))
                trace = pm.sample_smc(samples, cores=6, parallel=True)

            return trace
            print(function_pm)
        
    
    else:
        print('N layers incorrecto')